In [142]:
import numpy as np
import pandas as pd
import urllib.request, json 

In [143]:
# df = pd.read_csv('./us-states.csv')
# df = df[df.date == max(df.date)]
# df = df[(df.state != 'Guam') & (df.state != 'Puerto Rico') & (df.state != 'Virgin Islands')]
# df.drop(['fips', 'date'], axis=1, inplace=True)
# df.columns = ['State', 'Infected', 'Deaths']
# df.head(3)

### Live Infected, Deaths, COVID19 Testing data
(could maybe use this data for number of infections and deaths per state)  
http://coronavirusapi.com/states.csv

In [144]:
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

In [145]:
df = pd.read_csv('http://coronavirusapi.com/states.csv')
df['name,'] = df['name,'].apply(lambda r: states[r])
df.columns = ['State', 'Tested', 'Infected', 'Deaths']
df.head()

,State,Tested,Infected,Deaths
0,Alaska,3334,114,3.0
1,Alabama,4755,830,4.0
2,Arkansas,3536,449,6.0
3,Arizona,1303,912,16.0
4,California,89600,4643,101.0


### Gini coefficient

In [146]:
df = df.merge(pd.read_csv('./predictor_data/gini.csv'), on='State', how='inner')

### Total Population, ICU beds, percentage and amount age 60+

In [147]:
cols_of_interest = ['state', 'Total_pop', 'all_icu', '60plus', '60plus_per_each_icu_bed']
df_icu = pd.read_excel('./predictor_data/icu_beds.xlsx', usecols=cols_of_interest)
df_icu = df_icu.groupby('state').sum().reset_index()
df_icu['60plus_pct'] = df_icu['60plus'] / df_icu['Total_pop']
df_icu.columns = ['State', 'ICU Beds', 'Total Population', '60plus', '60plus per bed', '60plus pct pop']
df_icu.head(3)

,State,ICU Beds,Total Population,60plus,60plus per bed,60plus pct pop
0,Alabama,1533,4850771,1065625,53854.0,0.219682
1,Alaska,119,738565,117047,6424.0,0.158479
2,Arizona,1559,6809946,1502688,21304.0,0.220661


In [148]:
df = df.merge(df_icu, on='State', how='inner')

### Income per capita !!!!!!!!!!!!!!!!! change capitalization on this and GDP

In [149]:
df = df.merge(pd.read_csv('./predictor_data/income_per_capita.csv'), on='State', how='inner')
df.shape

(51, 11)

### GDP

In [150]:
df = df.merge(pd.read_csv('./predictor_data/state_gdp.csv'), on='State', how='inner')

### Unemployment

In [151]:
df = df.merge(pd.read_csv('./predictor_data/unemployment.csv'), on='State', how='inner')

### Sex

In [152]:
df_sex = pd.read_csv('./predictor_data/sex.csv')
df_sex['Sex Ratio'] = df_sex.Male / df_sex.Female 
df_sex = df_sex[['Location', 'Sex Ratio']]
df_sex.columns = ['State', 'Sex Ratio']

In [153]:
df = df.merge(df_sex, on='State', how='inner')

### Smoking

In [154]:
df_smoke = pd.read_csv('./predictor_data/smoking.csv')
df_smoke.columns = ['State', 'Smoking Rate']

In [155]:
df = df.merge(df_smoke, on='State', how='inner')

### Median Age

In [156]:
df_age = pd.read_csv('./predictor_data/age.csv')
df_age = df_age[['State', 'MedianAge', 'MedianAgeMale', 'MedianAgeFemale']]
df_age.columns = ['State', 'Median Age','Median Age M', 'Median Age F']

In [157]:
df = df.merge(df_age, on='State', how='inner')

### Flu and Pneumonia

In [158]:
df_flu = pd.read_csv('./predictor_data/Influenza_Pneumonia Mortality by State.csv')
df_flu = df_flu[df_flu.YEAR == 2018]
df_flu = df_flu[['STATE', 'RATE']]
df_flu.columns = ['State', 'Flu Deaths']
df_flu['State'] = df_flu['State'].apply(lambda r: states[r])
dc_df = pd.DataFrame([['District of Columbia', 11.2]], columns=['State', 'Flu Deaths']) # https://www.kff.org/other/state-indicator/influenza-and-pneumonia-death-rate/
df_flu = df_flu.append(dc_df, ignore_index=True)

In [159]:
df = df.merge(df_flu, on='State', how='inner')

### Chronic Lower Respiratory Disease Death Rate

In [160]:
df_resp = pd.read_csv('./predictor_data/Chronic Lower Respiratory Disease Mortality by State.csv')
df_resp = df_resp[df_resp.YEAR == 2018]
df_resp = df_resp[['STATE', 'RATE']]
df_resp.columns = ['State', 'Respiratory Deaths']
df_resp['State'] = df_resp['State'].apply(lambda r: states[r])
dc_df = pd.DataFrame([['District of Columbia', 19.6]], columns=['State', 'Respiratory Deaths']) # https://www.cdc.gov/nchs/pressroom/states/dc/dc.htm
df_resp = df_resp.append(dc_df, ignore_index=True)

In [161]:
df = df.merge(df_resp, on='State', how='inner')

### Doctors

In [162]:
df_doc = pd.read_csv('./predictor_data/doctors.csv')
df_doc = df_doc[['Location', 'Total']]
df_doc.columns = ['State', 'Physicians']

In [163]:
df = df.merge(df_doc, on='State', how='inner')

### Number Hospitals

In [164]:
df_hosp = pd.read_csv('./predictor_data/hospitals.csv')
df_hosp.columns = ['State', 'Hospitals']

In [165]:
df = df.merge(df_hosp, on='State', how='inner')

### Health Spending per capita

In [166]:
df_spend = pd.read_csv('./predictor_data/health_spending.csv')
df_spend.columns = ['State', 'Health Spending']

In [167]:
df = df.merge(df_spend, on='State', how='inner')

### Pollution

In [168]:
pollution_url = 'https://www.americashealthrankings.org/api/v1/measures/metrics/201?measure_id=147'
with urllib.request.urlopen(pollution_url) as url:
    data = json.loads(url.read().decode())
pollution = [{'State': states[i['StateCode']], 'Pollution':float(i['Value'])} for i in data['items'] if i['StateCode'] != 'ALL']
df_pol = pd.DataFrame(pollution)

In [169]:
df = df.merge(df_pol, on='State', how='inner')

### Airport

In [170]:
df_air = pd.read_csv('./predictor_data/airports.csv')
df_air['State'].value_counts()
df_air = df_air['State'].value_counts().rename_axis('State').reset_index(name='Med-Large Airports')
df_air['State'] = df_air['State'].apply(lambda r: states[r])
df_air.shape

(34, 2)

In [171]:
df = df.merge(df_air, on='State', how='left')
df = df.fillna(0)

### Avg Temperature

In [173]:
df = df.merge(pd.read_csv('./predictor_data/temperature.csv'), on='State', how='inner')

### 

In [174]:
df.to_csv('COVID19_state.csv', index=False)
df.head(51)

,State,Tested,Infected,Deaths,Gini,ICU Beds,Total Population,60plus,60plus per bed,60plus pct pop,...,Median Age M,Median Age F,Flu Deaths,Respiratory Deaths,Physicians,Hospitals,Health Spending,Pollution,Med-Large Airports,Avg Temp
0,Alaska,3334,114,3.0,0.4081,119,738565,117047,6424.0,0.158479,...,33.6,34.5,12.1,35.3,1900,21,11064,6.4,1.0,26.60
1,Alabama,4755,830,4.0,0.4847,1533,4850771,1065625,53854.0,0.219682,...,37.4,40.3,21.4,58.0,12205,101,7281,8.1,1.0,62.80
2,Arkansas,3536,449,6.0,0.4719,732,2977944,655552,27536.0,0.220136,...,36.6,39.3,18.0,61.7,7150,88,7408,7.1,0.0,60.40
3,Arizona,1303,912,16.0,0.4713,1559,6809946,1502688,21304.0,0.220661,...,36.1,38.7,12.4,41.2,17806,83,6452,9.7,1.0,60.30
4,California,89600,4643,101.0,0.4899,7338,38982847,7292299,68758.0,0.187064,...,35.2,37.5,15.6,30.9,112906,359,7549,12.8,9.0,59.40
5,Colorado,14470,2307,47.0,0.4586,1597,5436519,1031196,33639.0,0.189679,...,35.7,37.6,9.7,44.0,14509,89,6804,6.7,1.0,45.10
6,Connecticut,11900,1993,34.0,0.4945,674,3594478,805545,15436.0,0.224106,...,39.1,42.6,14.8,28.9,15605,32,9859,7.2,1.0,49.00
7,District of Columbia,3085,401,9.0,0.5420,314,672391,113026,360.0,0.168096,...,33.8,34.1,11.2,19.6,7346,10,11944,9.8,0.0,54.65
8,Delaware,268,232,6.0,0.4522,186,943732,220732,4103.0,0.233893,...,38.5,42.0,13.1,40.4,3148,7,10254,8.3,0.0,55.30
9,Florida,50528,4950,60.0,0.4852,5604,20278447,5200132,52222.0,0.256436,...,40.3,43.5,9.6,37.0,56499,217,8076,7.4,7.0,70.70
